In [51]:
# 什麼是Pod
! kubectl explain pod

KIND:     Pod
VERSION:  v1

DESCRIPTION:
     Pod is a collection of containers that can run on a host. This resource is
     created by clients and scheduled onto hosts.

FIELDS:
   apiVersion	<string>
     APIVersion defines the versioned schema of this representation of an
     object. Servers should convert recognized schemas to the latest internal
     value, and may reject unrecognized values. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#resources

   kind	<string>
     Kind is a string value representing the REST resource this object
     represents. Servers may infer this from the endpoint the client submits
     requests to. Cannot be updated. In CamelCase. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#types-kinds

   metadata	<Object>
     Standard object's metadata. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#met

In [49]:
# 透過kubectl run 建立一個Pod, 
# Pod 的名稱是 hostname,
# image 是用 docker.io/brokenpen/hostname:v1，
# labels 有 app=nginx, env=dev
! kubectl run hostname \
  --image=docker.io/brokenpen/hostname:v1 \
  --labels="app=nginx,env=dev"

pod/hostname created


In [41]:
# 取得目前 hostname 這個pods 和他的labels
! kubectl get pods hostname --show-labels
# 等5秒鍾
! sleep 5; echo; echo; echo;
# 透過 -o wide, 查看這個pods 的 IP 和 是在哪個節點上運行
! kubectl get pods hostname -o wide;
! echo; echo;

NAME       READY   STATUS    RESTARTS   AGE    LABELS
hostname   1/1     Running   0          2m4s   app=nginx,env=dev



NAME       READY   STATUS    RESTARTS   AGE    IP           NODE       NOMINATED NODE   READINESS GATES
hostname   1/1     Running   0          2m9s   172.17.0.6   minikube   <none>           <none>




In [52]:
# 進階的輸出方式, 不要 header, 欄位只要 :status.podIP
! kubectl get pods  \
  --no-headers \
  -o custom-columns=":status.podIP"

172.17.0.6
172.17.0.5
172.17.0.4
172.17.0.3


In [16]:
# ! curl $(kubectl get pod --no-headers -o custom-columns=":status.podIP"):8080

curl: (7) Failed to connect to 172.17.0.9 port 8080 after 3076 ms: No route to host


In [48]:
%%bash --bg --proc bg_proc
kubectl port-forward pod/hostname 8000:8080

In [44]:
! curl localhost:8000

You've hit hostname


In [45]:
! ss -tulpn | grep :8000

tcp   LISTEN 0      4096       127.0.0.1:8000       0.0.0.0:*    users:(("kubectl",pid=188817,fd=8)) 
tcp   LISTEN 0      4096           [::1]:8000          [::]:*    users:(("kubectl",pid=188817,fd=9)) 


In [46]:
# 把這個 kubectl port-forward pod/hostname 8080:8080 刪掉
# ! ss -tulpn | grep :8000  | awk -F, '{print $2 }' | awk -F= '{system("kill -9 " $2) }'

In [47]:
! kubectl delete pod/hostname

pod "hostname" deleted
^C


In [31]:
! cat pod.yaml

apiVersion: v1
kind: Pod
metadata:
  labels:
    app: hostname
    env: dev
  name: hostname
spec:
  containers:
  - image: docker.io/brokenpen/hostname:v1
    name: hostname


In [32]:
! kubectl get pods 

No resources found in default namespace.


In [34]:
! kubectl apply -f pod.yaml
! echo;
! kubectl get pod

pod/hostname unchanged

NAME       READY   STATUS    RESTARTS   AGE
hostname   1/1     Running   0          32s


In [35]:
! kubectl get pod hostname --show-labels

NAME       READY   STATUS    RESTARTS   AGE   LABELS
hostname   1/1     Running   0          41s   app=hostname,env=dev


In [36]:
! kubectl describe pod hostname

Name:             hostname
Namespace:        default
Priority:         0
Service Account:  default
Node:             minikube/192.168.49.2
Start Time:       Mon, 28 Nov 2022 08:00:32 +0000
Labels:           app=hostname
                  env=dev
Annotations:      <none>
Status:           Running
IP:               172.17.0.3
IPs:
  IP:  172.17.0.3
Containers:
  hostname:
    Container ID:   docker://be4e5288554f6f82aec0b68aa268343b3780cd638609d2278c02159bb17f0bd3
    Image:          docker.io/brokenpen/hostname:v1
    Image ID:       docker-pullable://brokenpen/hostname@sha256:e964abbaf720ac36ae20beb6330e81413d97160c11273ef582957d33a0608bae
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Mon, 28 Nov 2022 08:00:33 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from kube-api-access-44lkw (ro)
Conditions:
  Type              Status
  Initial

In [ ]:
! kubectl delete pod --all

pod "hostname" deleted
